# Visualizing Data with `PerspectiveWidget`

`PerspectiveWidget` offers a powerful widget that allows you to display, query, transform, and visualize your data interactively within a Jupyter Notebook. Both technical and non-technical users can use `PerspectiveWidget` to interact with datasets with a minimal amount of code, while developers can take advantage of the simple and powerful API to create their own notebooks and Voila applications using `PerspectiveWidget`.

For an overview of Perspective's core concepts such as the Table, View, Schema, etc., see `table_tutorial.ipynb` in the [`jupyter-notebooks`](https://github.com/finos/perspective/tree/master/examples/jupyter-notebooks) example folder.

In [ ]:
from perspective import Table, PerspectiveWidget
from datetime import date, datetime
import pandas as pd

### Creating a `PerspectiveWidget`

To create a widget instance, simply call the PerspectiveWidget constructor with your dataset:

In [ ]:
# Use superstore.arrow as our example dataset
import requests

# Download the arrow
url = "https://unpkg.com/@jpmorganchase/perspective-examples@0.2.0-beta.2/build/superstore.arrow"
req = requests.get(url)
arrow = req.content

In [ ]:
# Create a table
table = Table(arrow)
view = table.view()
df = view.to_df()
display(df.shape, df.dtypes)

Once the widget is created, you can interact with it like any other `<perspective-viewer>`, using drag-and-drop controls to apply pivots, filters, sorts, etc.

In [ ]:
# Create a widget, which will default to showing a datagrid
widget = PerspectiveWidget(df)
widget

For more control over your dataset (if you need to update the data frequently, etc.), create a `perspective.Table` and pass its handle into the `PerspectiveWidget` container:

In [ ]:
# Create a table so we can index the dataset
table = Table(df, index="Row ID")
widget2 = PerspectiveWidget(table)
widget2

When the `table` instance is updated with new data, the widget will reflect the new data:

In [ ]:
# Widget will display new data when this cell is called
table.update({
    "Row ID": [1],
    "Order ID": ["new order!"]
})

### Configuring `PerspectiveWidget`

`PerspectiveWidget`'s initializer can be configured with the following options to control its output:

In [ ]:
# Apply pivots and aggregates, and specify columns to show
PerspectiveWidget(df, columns=["Sales", "Category"], row_pivots=["State", "City"], column_pivots=["Segment"], aggregates={"Category": "dominant"})

In [ ]:
# Sorts and filters
PerspectiveWidget(df, columns=["Order Date", "State", "Sales", "Profit"], sort=[["Order Date", "desc"]], filters=[["State", "==", "Texas"]])

In [ ]:
# Calculate expression columns
PerspectiveWidget(df, columns=["expression"], expressions=['// expression\n"Sales" * "Profit" * (sqrt(144))'])

In [ ]:
# Show a chart by default
PerspectiveWidget(df, plugin="Y Bar", columns=["expression"], row_pivots=["State"], expressions=['//expression\n"Sales" / "Profit"'], sort=[["expression", "desc"]])

`save()` and `restore()` can be called to serialize a `PerspectiveWidget`'s entire state, and restore that state later on:

In [ ]:
# Create a complex widget
w = PerspectiveWidget(
    df,
    plugin="Heatmap",
    columns=["Sales"],
    row_pivots=["State"],
    sort=[["Sales", "desc"]]
)

config = w.save()

# Restore the config
PerspectiveWidget(df, **config)

### Using `PerspectiveWidget` with `ipywidgets`

Because `PerspectiveWidget` is itself an `ipywidget`, it can be easily included in any custom widget layouts and inside `ipywidget` elements:

In [ ]:
import ipywidgets as widgets

w1 = PerspectiveWidget(df, **config)
w2 = PerspectiveWidget(df, **config)

w2.columns = ["Profit"]
w2.sort = [["Profit", "desc"]]

# Create a tab widget with some PerspectiveWidgets inside
tab = widgets.Tab()
tab.children = [w1, w2]
tab.titles = ["Sales by State", "Profit by State"]
tab

`PerspectiveWidget` also works well with Jupyterlab's built in layout system - right click on any widget and click `Create New View for Output`, and you can resize and reposition that widget to anywhere inside the Jupyterlab window.

For more complex layouts that leverage the high performance and throughput of `perspective-python`, `perspective-workspace` allows the creation of powerful and complex front-end layouts, and integrates perfectly with `perspective-python` through the use of a Tornado server. For more details, see [this example](https://github.com/finos/perspective/tree/master/examples/workspace-editing-python).